# Algorithm Implement

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import quantstats as qs
import pandas as pd
import numpy as np
import sys
time_period = 2
sys.path.append('./')
from utlis import get_data, Stock_Env
import warnings
warnings.filterwarnings('ignore')
from collections import deque
    
class Q_Network(nn.Module):

    def __init__(self, state_size, action_size, N, hidden=[64, 64]):
        super(Q_Network, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden[0])
        self.fc2 = nn.Linear(hidden[0], hidden[1])
        self.fc3 = nn.Linear(hidden[1], action_size*N)

        self.action_size = action_size
        self.N = N

    def forward(self, state):
        x = state #(batch_size, state_size)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) #(batch_size, action_size*N)
        x = x.view(-1, self.action_size, self.N) #(batch_size, action_size, N)

        return x

# Data Loading

In [2]:
stock_df_train, stock_df_test, stock_df_train_, stock_df_test_, codes = get_data()

100%|███████████████████████████████████████████████████████████████████████████████| 251/251 [00:00<00:00, 488.46it/s]


In [3]:
codes_dict = dict(zip(codes, range(len(codes))))

# RL Model

In [4]:
import random
from collections import deque
import torch
import torch.optim as optim
import numpy as np

class Agent:

    def __init__(self, state_size, action_size, bs, lr, tau, gamma, N, kappa, device, visual=False):
        '''
        When dealing with visual inputs, state_size should work as num_of_frame
        '''
        self.state_size = state_size
        self.action_size = action_size
        self.bs = bs
        self.lr = lr
        self.tau = tau
        self.gamma = gamma
        self.device = device
        self.N = N
        self.tau = torch.linspace(0, 1, N+1)
        self.tau = (self.tau[:-1] + self.tau[1:]) / 2
        self.tau_hat = self.tau.to(device).unsqueeze(0) #(1, N)
        self.tau = tau
        self.kappa = kappa

        self.Q_local = Q_Network(self.state_size, self.action_size, N).to(self.device)
        self.Q_target = Q_Network(self.state_size, self.action_size, N).to(self.device)
        self.soft_update(1)
        self.optimizer = optim.Adam(self.Q_local.parameters(), self.lr)
        self.memory = deque(maxlen=100000)

    def act(self, state, eps=0):
        if random.random() > eps:
            state = torch.tensor(state, dtype=torch.float32).to(self.device)
            with torch.no_grad():
                action_values = self.Q_local(state)
                action_values = action_values.mean(dim=-1, keepdim=False).view(-1)
            return np.argmax(action_values.cpu().numpy())
        else:
            return random.choice(np.arange(self.action_size))

    def learn(self):
        experiences = random.sample(self.memory, self.bs)
        states = torch.from_numpy(np.vstack([e[0] for e in experiences])).float().to(self.device)
        actions = torch.from_numpy(np.vstack([e[1] for e in experiences])).long().to(self.device)
        rewards = torch.from_numpy(np.vstack([e[2] for e in experiences])).float().to(self.device)
        next_states = torch.from_numpy(np.vstack([e[3] for e in experiences])).float().to(self.device)
        dones = torch.from_numpy(np.vstack([e[4] for e in experiences]).astype(np.uint8)).float().to(self.device)

        quantiles_local = self.Q_local(states) #(batch_size, action_size, N)
        quantiles_local = torch.gather(input=quantiles_local, dim=1, index=actions.unsqueeze(1).repeat(1, 1, self.N)) #(batch_size, 1, N)

        with torch.no_grad():
            quantiles_target = self.Q_target(next_states) #(batch_size, action_size, N)
            next_actions = torch.max(quantiles_target.sum(dim=2, keepdim=True), dim=1, keepdim=True)[1] #(batch_size, 1, 1)
            quantiles_target = torch.gather(input=quantiles_target, index=next_actions.repeat(1,1,self.N), dim=1) #(batch_size, 1, N)
            quantiles_target = rewards.unsqueeze(1).repeat(1, 1, self.N) + self.gamma * (1 - dones.unsqueeze(1).repeat(1, 1, self.N)) * quantiles_target #(batch_size, 1, N)
            quantiles_target = quantiles_target.permute(0, 2, 1) #(batch_size, N, 1)

        diff = quantiles_target - quantiles_local #(batch_size, N, N)
        tau = self.tau_hat.unsqueeze(0).repeat(diff.size(0), 1, 1) #(batch_size, 1, N)
        loss = (tau - (diff<0).float()).abs() * self.huber(diff) #(batch_size, N, N)
        loss = loss.mean(dim=2, keepdim=False).sum(dim=1, keepdim=False) #(batch_size,)
        loss = loss.mean()

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        self.soft_update(self.tau)

    def huber(self, u):
        if self.kappa > 0:
            flag = (u.abs()<self.kappa).float()
            loss = 0.5*u.pow(2) * flag + self.kappa*(u.abs()-0.5*self.kappa) * (1-flag)
        else:
            loss = u.abs()
        return loss


    def soft_update(self, tau):
        for target_param, local_param in zip(self.Q_target.parameters(), self.Q_local.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)

In [5]:
import warnings
warnings.filterwarnings('ignore')
#env = gym.make()
num_episode = 500
max_t = 1000
reward_log = []
average_log = [] # monitor training process
eps = 1
eps_decay = 0.997
eps_min = 0.01
C = 4 # update weights every C steps

def validation(env, agent):
    rewards_log = []
    average_log = []
    episodic_reward = 0
    done = False
    t = 0
    state = env.reset()
    while not done and t < max_t:
        t += 1
        action = agent.act(state, eps)
        frame, reward, done = env.step(action)
        next_state = frame
        state = next_state.copy()
        episodic_reward += reward
        rewards_log.append(reward)
    sharpe = qs.stats.sharpe(pd.DataFrame(rewards_log))
    return env.asset, episodic_reward, sharpe

def train(env, agent, num_episode, eps_init, eps_decay, eps_min, max_t, num_frame=1, constant=0):
    rewards_log = []
    average_log = []
    state_history = []
    action_history = []
    done_history = []
    reward_history = []
    validation_log = []
    validation_average_log = []
    sharpe_log = []
    average_sharpe = []
    eps = eps_init

    for i in range(1, 1 + num_episode):

        episodic_reward = 0
        done = False
        frame = env.reset()
        state_deque = deque(maxlen=num_frame)
        for _ in range(num_frame):
            state_deque.append(frame)
        state = np.stack(state_deque, axis=0)
        state = np.expand_dims(state, axis=0)
        t = 0

        while not done and t < max_t:

            t += 1
            action = agent.act(state, eps)
            frame, reward, done = env.step(action)
            state_deque.append(frame)
            next_state = np.stack(state_deque, axis=0)
            next_state = np.expand_dims(next_state, axis=0)
            agent.memory.append((state, action, reward, next_state, done))

            if t % 5 == 0 and len(agent.memory) >= agent.bs:
                agent.learn()
                agent.soft_update(agent.tau)

            state = next_state.copy()
            episodic_reward += reward
        
        rewards_log.append(episodic_reward)
        average_log.append(np.mean(rewards_log[-100:]))
        val_asset, val_reward, val_sharpe = validation(env_test, agent)

        validation_log.append(val_reward)
        validation_average_log.append(np.mean(validation_log[-100:]))
        sharpe_log.append(val_sharpe.values[0])
        average_sharpe.append(np.mean(sharpe_log[-100:]))
        print('\rEpisode {}, Reward {:.3f}, Average Reward {:.3f}, valReward {:.3f}, val Average Reward {:.3f}, Asset {:.2f}, Validation Asset {:.2f}, Average Validation Sharpe {:.2f}'.format(i, episodic_reward, average_log[-1], val_reward, validation_average_log[-1], env.asset, val_asset, average_sharpe[-1]), end='')
        if i % 100 == 0:
            print()

        eps = max(eps * eps_decay, eps_min)

    return rewards_log

In [6]:
for code in codes:
    print(code, ' Begins')
    print('---------------------------------------------')
    env = Stock_Env(1000000, stock_df_train, 0.001, time = [x[0] for x in stock_df_train.index], record = stock_df_train_, codes_dict=codes_dict, train=True, code=code, time_period = time_period, codes=codes)
    env_test = Stock_Env(1000000, stock_df_test, 0.001, time = [x[0] for x in stock_df_test.index], record = stock_df_test_, codes_dict=codes_dict, train=False, code=code, time_period = time_period,  codes=codes)
    agent = Agent(2*5, env.action_space, 64, 0.001, 0.001, 0.99, 51, 1, 'cuda', True)
    train(env, agent, num_episode, eps, eps_decay, eps_min, max_t, num_frame=1, constant=C)

AAPL  Begins
---------------------------------------------
Episode 100, Reward 0.221, Average Reward 0.033, valReward 0.419, val Average Reward 0.271, Asset 1240190.76, Validation Asset 1502312.59, Average Validation Sharpe 1.95
Episode 200, Reward 0.118, Average Reward 0.086, valReward 0.254, val Average Reward 0.280, Asset 1121304.46, Validation Asset 1275956.42, Average Validation Sharpe 2.05
Episode 300, Reward 0.181, Average Reward 0.124, valReward 0.367, val Average Reward 0.262, Asset 1190947.60, Validation Asset 1427712.22, Average Validation Sharpe 1.90
Episode 400, Reward 0.312, Average Reward 0.149, valReward 0.280, val Average Reward 0.264, Asset 1357634.08, Validation Asset 1308648.77, Average Validation Sharpe 1.91
Episode 500, Reward 0.126, Average Reward 0.153, valReward 0.303, val Average Reward 0.275, Asset 1131579.34, Validation Asset 1341796.58, Average Validation Sharpe 2.02
NFLX  Begins
---------------------------------------------
Episode 100, Reward 0.007, Avera